In [3]:
! pip install openpyxl

In [9]:
import pandas as pd
import numpy as np

df = {}
df_full = pd.DataFrame()
for i in range(2008, 2023 + 1):
    df[i] = (pd.read_csv(f'../Flood/FloodData/Flood{i}.csv'))
    df[i]['Flood'] = False

In [11]:

#### 데이터 가공

df_flood_warning = pd.read_csv('./fldfct.csv', sep=',', header=None, skiprows=[0])
df_flood_warning.columns = ["홍수예보", "지점", "날짜", "상태"]
df_flood_warning.drop(['홍수예보'], axis=1, inplace=True)

mask = []
for data in df_flood_warning.iloc[:,-1].values:
    mask.append("발령" in data)
mask = np.array(mask)

df_flood_warning = df_flood_warning[mask]
df_flood_warning.sort_values('날짜', ascending=True, inplace=True)

mask = []
for data in df_flood_warning.loc[:, '지점'].values:
    mask.append("서울" in data or "한강대교" in data)
mask = np.array(mask)

df_flood_warning = df_flood_warning[mask]
df_flood_warning.sort_values('날짜', ascending=True, inplace=True)
df_flood_warning

,지점,날짜,상태
159,한강유역 한강대교지점,2011-07-28 02:00,홍수주의보 발령
130,한강(중랑천) 서울시(중랑교)지점,2016-07-05 10:30,홍수주의보 발령
118,한강 서울시(대곡교)지점,2017-07-16 04:50,홍수주의보 발령
112,한강 서울시(중랑교)지점,2018-08-28 20:30,홍수주의보 발령
102,한강 서울시(중랑교)지점,2018-08-30 00:20,홍수주의보 발령
75,한강 서울시(대곡교)지점,2020-08-06 05:50,홍수주의보 발령
73,한강 서울시(한강대교)지점,2020-08-06 11:00,홍수주의보 발령
43,한강 서울시(대곡교)지점,2022-06-30 08:20,홍수주의보 발령
37,한강 서울시(오금교)지점,2022-07-13 18:40,홍수주의보 발령
32,한강 서울시(중랑교)지점,2022-08-08 19:10,홍수주의보 발령


In [20]:
#### 데이터 구하기

startyear = 2008
maxyear = 2023
yearidx = startyear # 2008년부터 2023년 까지 증가

fidx = 0 # 0 ~ 홍수 데이터 끝까지
floodmax = len(df_flood_warning) # 홍수 데이터 끝 + 1 

idx = 0 # 0 ~ 해당 파일 끝까지 탐색

bfind = False # Nan 까지 완료했으면 True
starttime = str(df[yearidx]['일시'].iloc[idx])
for fidx in range(0, floodmax):
    strflood = str(df_flood_warning['날짜'].iloc[fidx])[:-3] # 홍수 첫 날짜
    flood_ymdh = strflood.split(' ')[0].split('-')
    flood_ymdh.append(strflood.split(' ')[1])
    flood_ymdh = list(map(int, flood_ymdh))
    
    idx = predidxstart = (flood_ymdh[1] - 1)*23*28 + (flood_ymdh[2] - 1)*23 + flood_ymdh[3] - 1
    
    while(True):
        strinfo = str(df[flood_ymdh[0]]['일시'].iloc[idx])[:-3] # 탐색
        info_ymdh = strinfo.split(' ')[0].split('-')
        info_ymdh.append(strinfo.split(' ')[1])
        info_ymdh = list(map(int, info_ymdh))
        if(info_ymdh == flood_ymdh):
            break
        idx += 1
    
    year = info_ymdh[0]
    while(True):
        if(idx >= 0):
            if(np.isnan(df[year].iloc[idx].loc["강수량(mm)"])):
                break
            else:
                df[year].loc[idx, "Flood"] = True
            idx -= 1
        else:
            year -= 1
            if(year < maxyear):
                break
            idx = len(df[year]) - 1

mask1 = (df[2022].Flood == True)
df_flood = df[2022].loc[mask1, :]
print(df_flood)


       지점 지점명                일시  기온(°C)  강수량(mm)  풍속(m/s)  풍향(16방위)  습도(%)   
4290  108  서울  2022-06-28 18:00    27.0      0.0      4.2       200     83  \
4291  108  서울  2022-06-28 19:00    27.1      0.0      5.6       200     83   
4292  108  서울  2022-06-28 20:00    27.2      0.0      5.3       200     81   
4293  108  서울  2022-06-28 21:00    26.3      0.8      3.9       200     87   
4294  108  서울  2022-06-28 22:00    26.8      0.0      3.5       200     82   
4295  108  서울  2022-06-28 23:00    26.3      0.3      3.0       180     85   
4296  108  서울  2022-06-29 00:00    25.7      0.8      3.5       180     89   
4297  108  서울  2022-06-29 01:00    25.3      0.9      3.7       200     93   
4298  108  서울  2022-06-29 02:00    24.8      7.8      3.5       180     95   
4299  108  서울  2022-06-29 03:00    24.8      1.3      6.1       200     96   
4300  108  서울  2022-06-29 04:00    25.1      0.4      8.8       200     95   
4301  108  서울  2022-06-29 05:00    24.9      0.9      4.2       